In [1]:
from sklearn.naive_bayes import BernoulliNB
import nltk
import string
from datasets import load_dataset
import re
from nltk.corpus import stopwords
from nltk import pos_tag
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import mlflow
from sklearnex import patch_sklearn
from warnings import filterwarnings
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
filterwarnings("ignore")

In [3]:
ag_news_dataset = load_dataset("IMDB")
stop_words = set(stopwords.words("english"))

In [4]:
text_preprocess_types = [None, 'стемминг', 'лемматизация']
#text_preprocess_types = ['лемматизация']

#words_classes = ['N', 'NJ', 'NJV', 'ALL']
words_classes = ['ALL']

#frequency_filtration_types = [None, 'low', 'high', 'both']
frequency_filtration_types = [None, 'low']

n_components = [150, 250, 375, 500, 750, 1000]

#hidden_layer_size = [375, 500, 750]

In [5]:
def base_ag_news_preprocess(text):
    tokens = text.lower()

    # Удаление спец слов
    special_words = ['<br /><br />'] # можно обработать римские цифры, удалить film, movie, ...
    for word in special_words:
        tokens = tokens.replace(word, '')
    
    # Удаление пунктуации и цифр
    #tokens = ''.join(i if i not in set(string.punctuation) - set('-') | set(string.digits) else ' ' for i in tokens)
    tokens = ''.join(i if i not in set(string.punctuation)  | set(string.digits) else ' ' for i in tokens)
    
    # Токенизация
    tokens = nltk.word_tokenize(tokens)
    
    # Удаление стоп слов
    stop_wordsL = stop_words - {'no','not'}
    #stop_wordsL = stop_words
    tokens = [word for word in tokens if (word not in stop_wordsL and word != '-')]
    return tokens

In [6]:
def different_ag_news_preprocess(tokens, preprocess_type, words_class):
    
    # Обработка слов
    if preprocess_type == 'лемматизация':
        lemmatizer = nltk.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    elif preprocess_type == 'стемминг':
        stemmer = nltk.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    
    # Обработка частей речи
    tokens = pos_tag(tokens)
    if words_class == 'N':
        tokens = [word for word, tag in tokens if tag.startswith('N')]
    elif words_class == 'NJ':
        tokens = [word for word, tag in tokens if tag.startswith('N') or tag.startswith('J')]
    elif words_class == 'NJV':
        tokens = [word for word, tag in tokens if tag.startswith('N') or tag.startswith('J') or tag.startswith('V')]
    elif words_class == 'ALL':
        tokens = [word for word, _ in tokens]
    
    return tokens  

In [7]:
def frequency_filtration(words_dictionary, frequency_filtration_type):
    if frequency_filtration_type == 'low':
        return dict([(key, value) for key, value in words_dictionary.items() if value >= 10 ])
    elif frequency_filtration_type == 'high':
        return dict([(key, value) for key, value in words_dictionary.items() if value <= 3000])
    elif frequency_filtration_type == 'both':
        return dict([(key, value) for key, value in words_dictionary.items() if 10 <= value <= 3000])
    else:
        return words_dictionary

In [8]:
def dummy(doc):
    return doc

In [9]:
def final_ag_news_preprocess(dataset, model_type):
    # Подготовка данных
    x_train = dataset['train']['text']
    y_train = dataset['train']['label']
    
    x_test = dataset['test']['text']
    y_test = dataset['test']['label']
    
    # Базовая обработка
    for i, text in enumerate(x_train):
        x_train[i] = base_ag_news_preprocess(text)
        
    for i, text in enumerate(x_test):
        x_test[i] = base_ag_news_preprocess(text)
    
    index = 0
    # Вариативная обработка
    for preprocess_type in text_preprocess_types: 
        for words_class in words_classes:         
            words = {}
            xtr = x_train
            xte = x_test
            
            # Обработка текстов
            for i, tokens in enumerate(xtr):
                final_tokens = different_ag_news_preprocess(tokens, preprocess_type, words_class)
                xtr[i] = final_tokens
                
                # Заполнение словаря
                for token in final_tokens:
                    if token not in words:
                        words[token] = 1
                    else:
                        words[token] += 1
            
            xte = [different_ag_news_preprocess(tokens, preprocess_type, words_class) for tokens in xte]
                    
            # Фильтрация по частоте
            for frequency_filtration_type in frequency_filtration_types: 
                filtered_words = frequency_filtration(words, frequency_filtration_type)
                token_length = len(filtered_words)
                
                # Векторизация слов
                word_list = sorted(filtered_words.keys())
                # Присвоение словам индексов
                words_indexed = {}
                for idx, word in enumerate(word_list):
                    words_indexed[word] = idx
                
                # TF-IDF
                vectorizer_TFIDF = TfidfVectorizer(vocabulary=words_indexed, preprocessor=dummy, tokenizer=dummy, dtype=np.float32)
                x_train_TFIDF = vectorizer_TFIDF.fit_transform(xtr)
                x_test_TFIDF = vectorizer_TFIDF.transform(xte)
                
                #LSI
                for n_num in n_components:
                    SVD_TFIDF = TruncatedSVD(n_components=n_num)
                    
                    SVD_TFIDF.fit(x_train_TFIDF)
                    
                    x_train_TFIDF_SVD = SVD_TFIDF.transform(x_train_TFIDF)
                    x_test_TFIDF_SVD = SVD_TFIDF.transform(x_test_TFIDF)

                    # Построение классификаторов
                    clf_B = BernoulliNB()
                    clf_B.fit(x_train_TFIDF_SVD, y_train)
                    prediction_B = clf_B.predict(x_test_TFIDF_SVD)
                    score_B = f1_score(y_test, prediction_B, average='macro')
                    
                    mlflow.start_run(run_name=f'Bernoulli_SVD{n_num}_TFIDF_{preprocess_type}_{words_class}_{frequency_filtration_type}')
                    mlflow.log_param('model', clf_B.__class__.__name__)
                    mlflow.log_param('preprocess_type', preprocess_type)
                    mlflow.log_param('words_class', words_class)
                    mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                    mlflow.log_param('token_length', token_length)
                    mlflow.log_param('n_components', n_num)
                    mlflow.log_param('word vectorizer', 'TFIDF')
                    mlflow.log_metric('macro_score', score_B)
                    mlflow.end_run()
                        
                    index += 1
                    print(f'Итерация {index}')
                
                clf_B = BernoulliNB()
                clf_B.fit(x_train_TFIDF, y_train)
                prediction_B = clf_B.predict(x_test_TFIDF)
                score_B = f1_score(y_test, prediction_B, average='macro')
                
                mlflow.start_run(run_name=f'Bernoulli_TFIDF_{preprocess_type}_{words_class}_{frequency_filtration_type}')
                mlflow.log_param('model', clf_B.__class__.__name__)
                mlflow.log_param('preprocess_type', preprocess_type)
                mlflow.log_param('words_class', words_class)
                mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                mlflow.log_param('token_length', token_length)
                #mlflow.log_param('n_components', n_num)
                mlflow.log_param('word vectorizer', 'TFIDF')
                mlflow.log_metric('macro_score', score_B)
                mlflow.end_run()

In [10]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name="IMDB_NB")

2025/04/08 21:26:51 INFO mlflow.tracking.fluent: Experiment with name 'IMDB_NB' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/267321236583644707', creation_time=1744136811774, experiment_id='267321236583644707', last_update_time=1744136811774, lifecycle_stage='active', name='IMDB_NB', tags={}>

In [11]:
final_ag_news_preprocess(ag_news_dataset, 'MLP')

🏃 View run Bernoulli_SVD150_TFIDF_None_ALL_None at: http://127.0.0.1:5000/#/experiments/267321236583644707/runs/506e9895f59f4bb9bdfa724b609d4562
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267321236583644707
Итерация 1
🏃 View run Bernoulli_SVD250_TFIDF_None_ALL_None at: http://127.0.0.1:5000/#/experiments/267321236583644707/runs/1591ac73fb2140e9aaeac8a33da36024
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267321236583644707
Итерация 2
🏃 View run Bernoulli_SVD375_TFIDF_None_ALL_None at: http://127.0.0.1:5000/#/experiments/267321236583644707/runs/a11b5f856cf84489b2a1c9a1bae998b5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267321236583644707
Итерация 3
🏃 View run Bernoulli_SVD500_TFIDF_None_ALL_None at: http://127.0.0.1:5000/#/experiments/267321236583644707/runs/5be291ad4ae84a3588ad694f4b08def9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/267321236583644707
Итерация 4
🏃 View run Bernoulli_SVD750_TFIDF_None_ALL_None at: http://127.0.0.1

Ручные тесты


In [10]:
dataset = ag_news_dataset
preprocess_type = 'лемматизация'
words_class = 'ALL'
frequency_filtration_type = 'low'

# Подготовка данных
x_train = dataset['train']['text']
y_train = dataset['train']['label']
    
x_test = dataset['test']['text']
y_test = dataset['test']['label']
    
# Базовая обработка
for i, text in enumerate(x_train):
    x_train[i] = base_ag_news_preprocess(text)
        
for i, text in enumerate(x_test):
    x_test[i] = base_ag_news_preprocess(text)
    
words = {}
xtr = x_train
xte = x_test
            
# Обработка текстов
for i, tokens in enumerate(xtr):
    final_tokens = different_ag_news_preprocess(tokens, preprocess_type, words_class)
    xtr[i] = final_tokens
                
    # Заполнение словаря
    for token in final_tokens:
        if token not in words:
            words[token] = 1
        else:
            words[token] += 1
            
xte = [different_ag_news_preprocess(tokens, preprocess_type, words_class) for tokens in xte]
                    
# Фильтрация по частоте
filtered_words = frequency_filtration(words, frequency_filtration_type)
token_length = len(filtered_words)
                
# Векторизация слов
word_list = sorted(filtered_words.keys())
# Присвоение словам индексов
words_indexed = {}
for idx, word in enumerate(word_list):
    words_indexed[word] = idx
    
# TF-IDF
vectorizer_TFIDF = TfidfVectorizer(vocabulary=words_indexed, preprocessor=dummy, tokenizer=dummy, dtype=np.float32)
x_train_TFIDF = vectorizer_TFIDF.fit_transform(xtr)
x_test_TFIDF = vectorizer_TFIDF.transform(xte)



In [119]:
SVD = TruncatedSVD(n_components=1000, random_state=42)

In [120]:
SVD.fit(x_train_TFIDF)

TruncatedSVD(n_components=1000, random_state=42)

In [121]:
x_train_SVD = SVD.transform(x_train_TFIDF)
x_test_SVD = SVD.transform(x_test_TFIDF)

In [131]:
clf = GaussianNB()
clf = clf.fit(x_train_TFIDF.toarray(), y_train)

In [133]:
predictions = clf.predict(x_test_TFIDF.toarray())
macro_score = f1_score(y_test, predictions, average='macro')

In [134]:
print(macro_score)

0.8333529656308173
